#라이브러리 로드



In [0]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --   upgrade pip
!pip install kaggle==1.5.6

! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16
!pip install kmc2




-rw-r--r-- 1 root root 62 Dec 14 11:24 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade
     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=ccde72e23746d89c47925ee697a8886d1a5c5a19f28d862551d4e6be1251b79a
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (

In [0]:
%matplotlib inline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import imutils 
import cv2 
import os
import kmc2
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cupy as xp 

import seaborn as sns
sns.set()

from sklearn.svm import SVC
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn.cluster import KMeans


In [0]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v


Kaggle API 1.5.6


In [0]:
! kaggle competitions download -c 2019-ml-finalproject
! unzip 2019-ml-finalproject.zip

 40% 26.0M/65.0M [00:01<00:02, 16.2MB/s]
100% 65.0M/65.0M [00:01<00:00, 48.0MB/s]
Archive:  2019-ml-finalproject.zip
  inflating: Label2Names.csv         
  inflating: sample_submission.csv   
  inflating: testAll_v2/image_0001.jpg  
  inflating: testAll_v2/image_0002.jpg  
  inflating: testAll_v2/image_0003.jpg  
  inflating: testAll_v2/image_0004.jpg  
  inflating: testAll_v2/image_0005.jpg  
  inflating: testAll_v2/image_0006.jpg  
  inflating: testAll_v2/image_0007.jpg  
  inflating: testAll_v2/image_0008.jpg  
  inflating: testAll_v2/image_0009.jpg  
  inflating: testAll_v2/image_0010.jpg  
  inflating: testAll_v2/image_0011.jpg  
  inflating: testAll_v2/image_0012.jpg  
  inflating: testAll_v2/image_0013.jpg  
  inflating: testAll_v2/image_0014.jpg  
  inflating: testAll_v2/image_0015.jpg  
  inflating: testAll_v2/image_0016.jpg  
  inflating: testAll_v2/image_0017.jpg  
  inflating: testAll_v2/image_0018.jpg  
  inflating: testAll_v2/image_0019.jpg  
  inflating: testAll_v2/imag

#함수


In [0]:
def dense_sitf_each(gray):
  sift=cv2.xfeatures2d_SIFT.create()

  keypoints=[]
  w, h = np.array(gray).shape
  for i in range(4, h, 8):
    for j in range(4, w, 8):
      keypoints.append(cv2.KeyPoint(i, j, 8))


  kp,des=sift.compute(gray, keypoints)
  
  return kp, des

#데이터 로드 및 분류





In [0]:

df_data= pd.read_csv('/content/Label2Names.csv',header=None)


In [0]:
DATA_ROOT_TRAIN="./train"
train_set=list()
train_labels=list()
train_des=list()



for cls in tqdm(os.listdir(DATA_ROOT_TRAIN)):
  img_list=os.listdir(DATA_ROOT_TRAIN+'/'+cls)

  if cls=='BACKGROUND_Google':
    label=102
    
   
  else:
    label=(df_data.index[df_data[1]==cls]+1).tolist()[0]
   

  for img in img_list:
    image=cv2.imread(DATA_ROOT_TRAIN+'/'+cls+'/'+img)
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
      

    kp,des=dense_sitf_each(gray)

    train_des.append(des)
    train_labels.append(label)





   
#set이랑  gray를 활용하는 모델은?  11/25 -> sift로 일단 데이터 전처리 11/26
#for 들여쓰기 조심

  
    



  

100%|██████████| 102/102 [02:25<00:00,  1.36s/it]


In [0]:
train_labels

In [0]:
#stack all the descriptors vertically in a numpy array
#descriptors=train_des[0]
#for descriptor in tqdm(train_des[1:]):
#descriptors=np.vstack((descriptors,descriptor))
descriptors= np.vstack(des for des in tqdm(train_des))



  0%|          | 0/3060 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.
100%|██████████| 3060/3060 [00:00<00:00, 693423.21it/s]


In [0]:
descriptors.shape

(3354868, 128)

In [0]:
from sklearn.cluster import MiniBatchKMeans
codebooksize=200

seeding = kmc2.kmc2(descriptors.reshape(-1,128), codebooksize)
Kmeas = MiniBatchKMeans(codebooksize, init=seeding).fit(descriptors.reshape(-1,128))
codebook = Kmeas.cluster_centers_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  """
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=400. Setting it to 3*k
  init_size=init_size)


In [0]:
print(codebook.shape)

(200, 128)


In [0]:
from scipy.cluster.vq import vq

hist=[]

for i in tqdm(range(len(train_des))):
  data=train_des[i]
  words, _=vq(data, codebook)
  word_hist,_=np.histogram(words,bins=range(codebooksize+1))
  
  hist.append(word_hist)

100%|██████████| 3060/3060 [00:11<00:00, 263.81it/s]


In [0]:
hist = np.asarray(hist)
train_labels = np.asarray(train_labels)

In [0]:
print(hist)
print(hist.shape)
print(train_labels.shape)

[[ 0  1  3 ...  0  0  5]
 [ 0  0  0 ... 23  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  2  0 ...  1  0  1]
 [ 0  0  0 ...  2  3  7]
 [ 0  0  2 ...  2  0  1]]
(3060, 400)
(3060,)


#test data



In [0]:
DATA_ROOT_TEST="./testAll_v2"
#test_set=list()
test_labels=list()
test_des=list()


img_list=os.listdir(DATA_ROOT_TEST)

for img in tqdm(img_list):
  image=cv2.imread(DATA_ROOT_TEST+'/'+img)
  gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

  kp,des=dense_sitf_each(gray)
  test_des.append(des)
  
  
  
  
  
 
  #image gray 활용을 어떻게 해야할지 생각 11/23-> 해결
  #11/21 데이터 로드 및  train,test data split
  #test set labeling 어떻게? ????? 11/23->  할  필요 없음 11/26





100%|██████████| 1692/1692 [01:22<00:00, 20.57it/s]


In [0]:
#stack all the descriptors vertically in a numpy array

#descriptors_test = test_des[0]
#for descriptor_test in tqdm(test_des[1:]):
  #descriptors_test=np.vstack((descriptors_test,descriptor_test))
descriptors_test = np.vstack(des for des in test_des)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


In [0]:
from scipy.cluster.vq import vq

hist_test=[]
for i in tqdm(range(len(test_des))):
  data=test_des[i]
  words,_=vq(data, codebook)
  word_hist,_=np.histogram(words,bins=range(codebooksize+1))
  hist_test.append(word_hist)

hist_test=np.asarray(hist_test)

100%|██████████| 1692/1692 [00:06<00:00, 249.25it/s]


#svm

In [0]:
from sklearn.model_selection import train_test_split
import cupy as xp
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA as RandomizedPCA

from sklearn.pipeline import make_pipeline


x_train, x_test, y_train, y_test = train_test_split(hist, train_labels)
svm = SVC(kernel='rbf',C=50,gamma=1e-5 ,class_weight='balanced').fit(x_train,y_train)





yfit = svm.predict(hist_test)


#출력 및 제출

In [0]:
result=yfit

In [0]:
result=result.reshape(-1,1)
result_img_list=np.array(img_list).reshape(-1,1)
total_result=np.hstack([result_img_list,result])

In [0]:
print(total_result)

[['image_1389.jpg' '8']
 ['image_0810.jpg' '21']
 ['image_1265.jpg' '12']
 ...
 ['image_0532.jpg' '27']
 ['image_0288.jpg' '38']
 ['image_0906.jpg' '75']]


In [0]:
df=pd.DataFrame(total_result,columns=['Id',"Category"])
df.to_csv('results-sungbin-v3.csv',index=False,header=True)


In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-sungbin-v3.csv -m "sung bin kim"

100% 29.6k/29.6k [00:06<00:00, 4.55kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 